In [ ]:
# 必要なものを用意
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

device = "cuda" if torch.cuda.is_available() else "cpu"
model_name = "gpt2-xl"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name).to(device)

max_length = 128
input_txt = """In a shocking finding, scientist discovered \
a herd of unicorns living in a remote, previously unexplored \
valley, in the Andes Mountains. Even more surprising to the \
researchers was the fact that the unicorns spoke perfect English.\n\n"""
input_ids = tokenizer(input_txt, return_tensors="pt")["input_ids"].to(device)

In [ ]:
# matplotlib での日本語表示
## 参考：https://qiita.com/naohiro2g/items/5cb79763a14e052db768
!pip install japanize-matplotlib
import japanize_matplotlib

In [ ]:
# copilot 製の確率可視化コード
import torch
import numpy as np
import matplotlib.pyplot as plt
from transformers import AutoTokenizer, AutoModelForCausalLM

# ===== 設定 =====
ja_model_name = "rinna/japanese-gpt2-small"
prompt = "今日の天気は"

temperature = 1.0
min_p, max_p = 1e-10, 1e-1   # 確率分布(対数)の表示範囲
num_bins = 80

k_threshold = 2000           # Top-k 閾値
p_threshold = 0.95           # Top-p 閾値
xmax_cdf = 10000             # 累積分布の横軸上限

# ===== モデル読み込み =====
ja_tokenizer = AutoTokenizer.from_pretrained(ja_model_name)
ja_model = AutoModelForCausalLM.from_pretrained(ja_model_name)
ja_model.eval()

inputs = ja_tokenizer(prompt, return_tensors="pt")

# ===== 次トークン確率（T=1） =====
with torch.no_grad():
    outputs = ja_model(**inputs)
    logits = outputs.logits[0, -1, :]
    probs = torch.softmax(logits / temperature, dim=-1).cpu().numpy()

# ===== 1) 確率分布（対数ヒストグラム） =====
plot_probs = probs[(probs >= min_p) & (probs <= max_p)]
log_bins = np.logspace(np.log10(min_p), np.log10(max_p), num_bins)
counts, edges = np.histogram(plot_probs, bins=log_bins)

# ===== 2) 累積分布（高確率順） =====
sorted_probs = np.sort(probs)[::-1]
cum_probs = np.cumsum(sorted_probs)
ranks = np.arange(1, len(sorted_probs) + 1)

# ===== 描画 =====
fig, ax = plt.subplots(1, 2, figsize=(13, 4.5))

# 左: 確率分布
ax[0].bar(edges[:-1], counts, width=np.diff(edges), align="edge")
ax[0].set_xscale("log")
ax[0].set_yscale("log")
ax[0].set_xlim(min_p, max_p)
ax[0].set_xlabel("確率")
ax[0].set_ylabel("カウント")
ax[0].set_title("確率分布")

# 右: 累積分布
ax[1].plot(ranks, cum_probs, lw=1.5)
ax[1].set_xlim(0, xmax_cdf)
ax[1].set_ylim(0, 1.01)
ax[1].set_xlabel("トークン(確率の高い順)")
ax[1].set_ylabel("確率")
ax[1].set_title("累積分布")

# 閾値線
ax[1].axvline(k_threshold, color="tab:orange", ls="-", lw=1.5, label=f"Top-k threshold: k={k_threshold}")
ax[1].axhline(p_threshold, color="tab:red", ls="--", lw=1.5, label=f"Top-p threshold: p={p_threshold}")
ax[1].legend(loc="lower right")

japanize_matplotlib.japanize()
plt.tight_layout()
plt.show()

In [ ]:
# Top-k サンプリング
output_topk = model.generate(input_ids, max_length=max_length, do_sample=True,
                             top_k=50)
print(tokenizer.decode(output_topk[0]))

In [ ]:
# Top-p サンプリング
output_topp = model.generate(input_ids, max_length=max_length, do_sample=True,
                             top_p=0.90)
print(tokenizer.decode(output_topp[0]))